In [1]:
import os
import os.path
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold                                                                                                                       
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import sklearn_evaluation.plot as skplot
from sklearn.model_selection import learning_curve

import matplotlib.pyplot as plt

import numpy as np
np.random.seed(1)

from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
from keras.preprocessing import image
from keras.applications.inception_resnet_v2 import InceptionResNetV2,preprocess_input

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
imagedir = "Datasets/ByFamilyPortexByteplot"

In [3]:
cur_dir = os.getcwd()
os.chdir(imagedir)  # the parent folder with sub-folders

# Get number of samples per family
list_fams = sorted(os.listdir(os.getcwd()), key=str.lower)  # vector of strings with family names
no_imgs = []  # No. of samples per family
for i in range(len(list_fams)):
    os.chdir(list_fams[i])
    len1 = len(glob.glob('*.png'))  # assuming the images are stored as 'png'
    no_imgs.append(len1)
    os.chdir('..')
num_samples = np.sum(no_imgs)  # total number of all samples

# Compute the labels
y = np.zeros(num_samples)
pos = 0
label = 0
for i in no_imgs:
    print ("Label:%2d\tFamily: %15s\tNumber of images: %d" % (label, list_fams[label], i))
    for j in range(i):
        y[pos] = label
        pos += 1
    label += 1
num_classes = label

# Compute the features
width, height,channels = (224,224,3)
X = np.zeros((num_samples, width, height, channels))
cnt = 0
list_paths = [] # List of image paths
print("Processing images ...")
for i in range(len(list_fams)):
    for img_file in glob.glob(list_fams[i]+'/*.png'):
        #print("[%d] Processing image: %s" % (cnt, img_file))
        list_paths.append(os.path.join(os.getcwd(),img_file))
        img = image.load_img(img_file, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        X[cnt] = x
        cnt += 1
print("Images processed: %d" %(cnt))

os.chdir(cur_dir)

Label: 0	Family:         allaple	Number of images: 362
Label: 1	Family:           alman	Number of images: 325
Label: 2	Family:          autoit	Number of images: 261
Label: 3	Family:            daws	Number of images: 466
Label: 4	Family:            delf	Number of images: 359
Label: 5	Family:         gamarue	Number of images: 259
Label: 6	Family:          ibryte	Number of images: 347
Label: 7	Family:          loring	Number of images: 285
Label: 8	Family:          mydoom	Number of images: 578
Label: 9	Family:          qukart	Number of images: 253
Label:10	Family:          ramnit	Number of images: 506
Label:11	Family:          sality	Number of images: 1401
Label:12	Family:          simbot	Number of images: 1148
Label:13	Family:       softpulse	Number of images: 912
Label:14	Family:          viking	Number of images: 183
Label:15	Family:         virlock	Number of images: 373
Label:16	Family:          vobfus	Number of images: 405
Label:17	Family:          wapomi	Number of images: 345
Label:18

In [4]:
X.shape

(10136, 224, 224, 3)

In [5]:
# Creating base_model
image_shape = (224, 224, 3)                                                                                                                                                                                                                                                                                            
base_model = InceptionResNetV2(weights='imagenet', input_shape=image_shape, include_top=False, pooling='avg')

In [6]:
print("Extracting bottleneck features from convolutional layers ...")
bnfeatures = base_model.predict(X)

Extracting bottleneck features from convolutional layers ...


In [7]:
bnfeatures.shape

(10136, 1536)

In [8]:
bnfeatures = np.reshape(bnfeatures,(bnfeatures.shape[0],-1))

In [9]:
bnfeatures.shape

(10136, 1536)

In [10]:
# Create stratified k-fold subsets                                                                                                                                        
kfold = 10  # no. of folds                                                                 
skf = StratifiedKFold(kfold, shuffle=True,random_state=1)

In [11]:
# Training top_model
C_range = np.logspace(-2, 10, 13)
param_grid = dict(C=C_range)

top_model = GridSearchCV(svm.LinearSVC(), param_grid=param_grid, cv=skf, verbose=10, n_jobs=-1)
top_model.fit(bnfeatures,y)  # Training 

Fitting 10 folds for each of 13 candidates, totalling 130 fits
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  3.5min


[CV] ........................... C=0.01, score=0.858556, total= 3.5min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.863095, total= 3.5min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.868186, total= 3.6min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.856299, total= 3.7min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.866928, total= 3.7min
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.862745, total= 3.7min
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.871769, total= 3.7min
[CV] C=10.0 ..........................................................
[CV] .

[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  8.1min


[CV] ............................ C=0.1, score=0.869436, total= 8.6min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.867647, total= 8.9min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.883053, total= 9.1min
[CV] C=10.0 ..........................................................
[CV] ............................ C=1.0, score=0.878906, total= 9.2min
[CV] C=100.0 .........................................................
[CV] ............................ C=1.0, score=0.853922, total= 9.2min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.875992, total= 9.3min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.879883, total= 9.4min
[CV] C=100.0 .........................................................
[CV] .

...

[CV] ................... C=1000000000.0, score=0.860119, total= 5.0min
[CV] .................... C=100000000.0, score=0.855865, total= 6.0min
[CV] ................... C=1000000000.0, score=0.876953, total= 5.7min
[CV] .................... C=100000000.0, score=0.856574, total= 6.0min
[CV] ................... C=1000000000.0, score=0.852941, total= 5.6min
[CV] ................... C=1000000000.0, score=0.867126, total= 5.5min
[CV] ................... C=1000000000.0, score=0.869863, total= 5.7min
[CV] .................. C=10000000000.0, score=0.853922, total= 4.1min
[CV] ................... C=1000000000.0, score=0.846686, total= 5.5min
[CV] ................... C=1000000000.0, score=0.859841, total= 5.3min
[CV] .................. C=10000000000.0, score=0.870079, total= 3.9min
[CV] .................. C=10000000000.0, score=0.865157, total= 3.9min
[CV] .................. C=10000000000.0, score=0.868885, total= 4.4min
[CV] .................. C=10000000000.0, score=0.849654, total= 3.8min


[Parallel(n_jobs=-1)]: Done 125 out of 130 | elapsed: 41.8min remaining:  1.7min


[CV] .................. C=10000000000.0, score=0.877930, total= 4.7min
[CV] .................. C=10000000000.0, score=0.869177, total= 3.4min
[CV] .................. C=10000000000.0, score=0.861111, total= 3.5min
[CV] .................. C=10000000000.0, score=0.859841, total= 3.3min
[CV] .................. C=10000000000.0, score=0.855578, total= 3.4min


[Parallel(n_jobs=-1)]: Done 130 out of 130 | elapsed: 42.5min finished


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=1, shuffle=True),
       error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05,
       1.e+06, 1.e+07, 1.e+08, 1.e+09, 1.e+10])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=10)

In [12]:
top_model.cv_results_

{'mean_fit_time': array([215.76752479, 535.68598921, 518.4837224 , 523.06747506,
        525.19716055, 529.09045315, 542.16349072, 528.70936432,
        539.20879967, 520.29887006, 440.9278861 , 311.9453254 ,
        230.80213401]),
 'mean_score_time': array([0.0648072 , 0.06536415, 0.05691431, 0.04297261, 0.03815565,
        0.04571171, 0.04611595, 0.05868347, 0.04344153, 0.02859967,
        0.02761531, 0.01909592, 0.01400466]),
 'mean_test_score': array([0.86069455, 0.87480268, 0.86799526, 0.8636543 , 0.86177979,
        0.86404893, 0.86177979, 0.86177979, 0.86276638, 0.86325967,
        0.86108919, 0.8624704 , 0.86316101]),
 'mean_train_score': array([0.92042673, 0.98183612, 0.99829022, 0.99778557, 0.99704098,
        0.9973909 , 0.99706274, 0.9981475 , 0.99845447, 0.99798304,
        0.99797174, 0.99839961, 0.99788513]),
 'param_C': masked_array(data=[0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0,
                    1000000.0, 10000000.0, 100000000.0, 1000000000.0,
      

In [13]:
top_model.grid_scores_

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.86069, std: 0.00660, params: {'C': 0.01},
 mean: 0.87480, std: 0.00626, params: {'C': 0.1},
 mean: 0.86800, std: 0.00884, params: {'C': 1.0},
 mean: 0.86365, std: 0.00821, params: {'C': 10.0},
 mean: 0.86178, std: 0.00915, params: {'C': 100.0},
 mean: 0.86405, std: 0.00832, params: {'C': 1000.0},
 mean: 0.86178, std: 0.00786, params: {'C': 10000.0},
 mean: 0.86178, std: 0.00861, params: {'C': 100000.0},
 mean: 0.86277, std: 0.00833, params: {'C': 1000000.0},
 mean: 0.86326, std: 0.00737, params: {'C': 10000000.0},
 mean: 0.86109, std: 0.00657, params: {'C': 100000000.0},
 mean: 0.86247, std: 0.00891, params: {'C': 1000000000.0},
 mean: 0.86316, std: 0.00824, params: {'C': 10000000000.0}]

In [14]:
print("Best parameters: %s - score: %0.4f" % (top_model.best_params_, top_model.best_score_))

Best parameters: {'C': 0.1} - score: 0.8748
